In [933]:
import numpy as np
import math
import itertools
import pandas as pd

# lowest priority, -1 to get # of different priorities
MAX_PRIO = 6

# deck size
M = 5

# hand size
K = 3

In [965]:
# Create the first column (index counting up from 101)
test_unit_id = np.arange(M) + 101

# Generate random priority
rng = np.random.default_rng()
prios = rng.integers(1, MAX_PRIO, size=M)

# Combine the columns into a single array
deck = pd.DataFrame(prios, index=test_unit_id, columns=["priority"])
deck.index.rename("unit_id", inplace=True)

priority_counts = deck.priority.value_counts().sort_index()
print(priority_counts)
deck

priority
1    1
2    2
4    1
5    1
Name: count, dtype: int64


,priority
unit_id,
101,2
102,1
103,4
104,2
105,5


In [994]:
played_card = 100
played_card_priority = 3  # deck.loc[played_card, "priority"]
print(played_card_priority)

all_hands = list(itertools.combinations(deck.index, K))
print(all_hands)

for hand in all_hands:
    count = 0
    for card in hand:
        if deck.loc[card, "priority"] < played_card_priority:
            count += 1
    print(hand, count)

filtered_hands = [hand for hand in all_hands if sum(deck.loc[card, "priority"] < played_card_priority for card in hand) <= 1]  # type: ignore
print(filtered_hands)

3
[(101, 102, 103), (101, 102, 104), (101, 102, 105), (101, 103, 104), (101, 103, 105), (101, 104, 105), (102, 103, 104), (102, 103, 105), (102, 104, 105), (103, 104, 105)]
(101, 102, 103) 2
(101, 102, 104) 3
(101, 102, 105) 2
(101, 103, 104) 2
(101, 103, 105) 1
(101, 104, 105) 2
(102, 103, 104) 2
(102, 103, 105) 1
(102, 104, 105) 2
(103, 104, 105) 1
[(101, 103, 105), (102, 103, 105), (103, 104, 105)]


In [936]:
# get probability of a single priority class being played
def prob(m_niedriger, m_i, m, k) -> float:
    return (1 / math.comb(m, k)) * (
        math.comb(m_i, k)
        + m_niedriger * math.comb(m_i, k - 1)
        + math.comb(m_niedriger, k - 1) * m_i
    )


# get list of chances for all priority classes
def getStochasticProb(deck, max_prio, m, k) -> list:
    stochastic_prob = []
    for prio in range(1, max_prio):
        stochastic_prob.append(
            prob(
                np.count_nonzero(deck["priority"] > prio),
                np.count_nonzero(deck["priority"] == prio),
                m,
                k,
            )
        )
    return stochastic_prob

def getPossibleHands(deck, k, played_card_priority) -> list:
    all_hands = list(itertools.combinations(deck.index, k))
    # TODO: temp
    return all_hands

    if played_card == 0:
        return all_hands
    
    
# execute recursive loop, demo playthrough
def all_chances(deck, max_prio, m, k, played_card=0) -> None:
    if len(deck) < 3:
        return

    # get latest # per priority class
    priority_counts = deck.priority.value_counts().sort_index()
    # print(priority_counts)

    # list of tuples of 3 unit_ids
    possible_hands = getPossibleHands(deck, k, played_card)
    # print(possible_hands)

    # list of floats, chance per priority class
    stochastic_prob = getStochasticProb(deck, max_prio, m, k)
    # print(stochastic_prob)

    # adjust probability for single cards
    deck["chance"] = deck["priority"].apply(
        lambda x: stochastic_prob[x - 1] / priority_counts.loc[x]
    )

    # rng initial and played hand cards
    chosen_hand = list(possible_hands[rng.integers(0, len(possible_hands))])
    # print(chosen_hand)

    top_priority = deck.loc[chosen_hand, "priority"].min()
    # print(top_priority)

    top_choices = deck[deck["priority"] == top_priority]
    # print(top_choices)

    played_card = rng.choice(top_choices.index)
    # print(played_card)

    print(deck.sort_values("chance", ascending=False), "\n")

    played_card_priority = deck.loc[played_card, "priority"]

    # clean up deck
    deck.drop(columns="chance", index=played_card, inplace=True)

    return all_chances(deck, max_prio, m - 1, k, played_card_priority)

In [937]:
all_chances(deck, MAX_PRIO, M, K)

         priority  chance
unit_id                  
104             1     0.6
102             3     0.2
105             3     0.2
101             5     0.0
103             4     0.0 

         priority  chance
unit_id                  
102             3     0.5
105             3     0.5
101             5     0.0
103             4     0.0 

         priority  chance
unit_id                  
105             3     1.0
101             5     0.0
103             4     0.0 

